<a href="https://colab.research.google.com/github/Adam129511/sieci2/blob/main/CNN_catsVSdogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##KONWOLUCYJNE SIECI NEURONOWE - Dogs vs Cats
Projekt polega na przyjrzeniu się sieciom konwolucyjnym i próbie zastosowania tych sieci w zagadnieniach klasyfikacji obrazu. Pokażemy jak postępować w przypadku niewielkiego zbioru danych treningowych i jakie techniki zastosować w celu optymalizacji takich modeli

W odróżnieniu od sieci gęstych, które uczą się cech globalnie, sieci konwolucyjne uczą się lokalnych wzorców. Innymi słowy wzorce rozpoznawane przez sieć konwolucyjną są niezależne od przesunięcia, co charakteryzuje sieci CNN bardzo dużą wydajnością.

Kolejną zaletą sieci CNN jest możliwość uczenia przestrzennej hierarchi wzorców. Pierwsze warstwy uczą się małych lokalnych wzorców, takich jak krawęzie, zaś kolejne warstwy będą uczyły się większych struktur składających się z elementów rozpoznanych przez początkowe warstwy.




#Czyszczenie katalogu roboczego


> Wpisując ba początku komórki polecenie %%bash jesteśmy w stanie wykonywać polecenia bashowe na systemie goszczącym naszą instancję notebook`a. Na początku wyczyścimy nasz katalog roboczy






In [1]:
%%bash
rm -rf /content/*

In [2]:
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

#Pobieranie danych

Dane pobierzemy z bucketu S3. S3 jest to usługa storage w chmurze Amazona.

Nazwa bucketu to: ml-repository-krakers.
Spakowany plik dogs-vs-cats.zip ma nieco ponad 812 MB i jest przechowywany w regionie UE(ireland)

In [3]:
%%bash
pwd #funkcja drukująca katalog roboczy w którym się znajdujemy
wget --output-document=cats_and_dogs.zip --quiet https://ml-repository-krakers.s3-eu-west-1.amazonaws.com/kaggle+/cats_and_dogs/dogs-vs-cats.zip #funckcja pobiera dane i zapisuje w pliku cats_and_dogs.zip
ls


/content
cats_and_dogs.zip


#Wypakowanie danych

In [4]:
%%bash
unzip cats_and_dogs.zip

rm -rf sampleSubmission.csv

unzip -q train.zip -d /content/kaggle_original_data
rm -rf train.zip
cp /content/kaggle_original_data/train/* /content/kaggle_original_data/
rm -rf /content/kaggle_original_data/train
unzip -q /content/test1.zip




Archive:  cats_and_dogs.zip
  inflating: sampleSubmission.csv    
  inflating: train.zip               
  inflating: test1.zip               


In [5]:
%%bash


ls


cats_and_dogs.zip
kaggle_original_data
test1
test1.zip


#Zdefiniowanie uzytecznych funkcji

In [7]:
def make_accuracy_plot(history):
     #Funkcja zwraca wykres dokładośći (accuracy) modelu na zbiorze treningowym i walidacyjnym
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set()
    acc, val_acc = history.history['accuracy'], history.history['val_accuracy']
    epochs = range(1, len(acc)+1)

    plt.figure(figsize = (10,8))
    plt.plot(epochs, acc, label = "dokładność trenowania", marker = 'o')
    plt.plot(epochs, val_acc, label = "dokładność walidacji", marker = 'o')
    plt.legend()
    plt.title('Dokładność trenowania i walidacji')
    plt.xlabel('epoki')
    plt.ylabel('Dokładność')
    plt.show()

def make_loss_plot(history): #Funkcja zwraca wykres straty (loss) na zbiorze treningowym i walidacyjnym
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set()
    loss, val_loss = history.history['loss'], history.history['val_loss']
    epochs= range(1, len(loss)+1)

    plt.figure(figsize = (10,8))
    plt.plot(epochs, loss, label = "Strata trenowania", marker = 'o')
    plt.plot(epochs, val_loss, label = "Strata walidacji", marker = 'o')
    plt.legend()
    plt.title('Strata trenowania i walidacji')
    plt.xlabel('epoki')
    plt.ylabel('Strata')
    plt.show()

def display_augmented_images(directory, index):#Funkcja zwraca wykres przykładowych obrazów za pomocą techniki augmentacji danych.

    fnames = [os.path.join(directory, fname) for fname in os.listdir(directory)] #pobranie ściezek do plików gdzie są zdjęcia
    img_path = fnames[index]
    img = image.load_img(img_path, target_size=(150,150))

    x = image.img_to_array(img)
    x = x.reshape((1,) + x.shape)

    i = 1
    plt.figure(figsize=(20,14))
    for batch in datagen.flow(x, batch_size=1):
        plt.subplot(3,4,i)
        plt.grid(False)
        imgplot = plt.imshow(image.array_to_img(batch[0]))
        i+=1
        if i % 13 == 0:
            break



##Przygotowanie katalogów
Nasze dane będą znajdować się w katalogu:

> /content/cats_and_dogs

podzielimy ten katalog na trzy podkatalogi:
*   train
*   valid
*   test

W zbiorze treningowym odłożymy 2000 próbek(1000 zdjęciów psów i 1000 zdjęć kotów), do zbioru walidacyjnego trafi 1000 próbek (500 zdjęć psów i 500 zdjęć kotów), zaś do zbioru testowego ostatnie 1000 próbek (500 zdjęć psów i 500 zdjęć kotów).









